In [ ]:
'''
AI Agent Chatbot
    - Langchain Tutos: https://python.langchain.com/docs/tutorials/chatbot/
'''

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from typing_extensions import Annotated, TypedDict
from langchain_community.utilities import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

In [3]:
# Model config
# model = OllamaLLM(model="llama3.2")

from langchain.chat_models import init_chat_model

model = init_chat_model("llama3.2", model_provider="ollama")

In [10]:
# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

In [21]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a kind artificial assistant. You are a helpful assistant. You try to respond to every question with helpful information. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [19]:
language = "English"

In [20]:
# Define the function that calls the model
def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}

# # Async function for node:
# async def call_model(state: MessagesState):
#     response = await model.ainvoke(state["messages"])
#     return {"messages": response}

In [12]:
# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# async 
# workflow = StateGraph(state_schema=MessagesState)
# workflow.add_edge(START, "model")
# workflow.add_node("model", call_model)

In [13]:
# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# async
# app = workflow.compile(checkpointer=memory)

In [16]:
thread = "abc123" # thread of the conversation
config = {"configurable": {"thread_id": thread }} 

In [ ]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

# Async invocation:
# output = await app.ainvoke({"messages": input_messages, "language": language}, config)
# output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I think I see a pattern here, Bob! Do you want to talk about something in particular, or would you like me to try and guess what's on your mind?


: 